### Применение бутстрепа в анализе данных по длительности поездки на такси.

#### Автор: Тушева Ксения

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from arch.bootstrap import IIDBootstrap, IndependentSamplesBootstrap

Для проведения процедуры бутстрепа используем подвыборку из 30 элементов (х - продолжительность поездки на такси), случайно взятых из генеральной совокупности. Генеральная совокупность представленна данными из соревнования [New York City Taxi Trip Duration](https://www.kaggle.com/c/nyc-taxi-trip-duration/overview).

In [6]:
# Случайно составим подвыборку х
df = pd.read_csv("/home/ksu/Загрузки/train.csv")
rng = np.random.default_rng(123)
x = rng.choice(df['trip_duration'], size=30)

In [7]:
x 

array([1165,  300,  556,  555,  939,  385,  259,  324, 1283, 1163,  177,
        810,  645,  386,  628,  763, 3717,  906,  469,  323,  734,  545,
       1274,  358,  453,  587,  510,  989,  470,  446])

Так как истинное распределение неизвестно - уместно использовать бутстреп

In [10]:
# Для примера сделаем первую фейковую подвыборку вручную
x_star = rng.choice(x, size=len(x))

In [11]:
x_star

array([ 734, 1163,  386, 1165,  545, 1165,  810,  628,  734,  628,  587,
        555,  324,  324,  810, 1165,  645,  810,  906,  906, 1283,  645,
        763,  587,  556,  469,  556,  324,  386,  453])

In [13]:
# Теперь воспользуемся пакетом для симуляции 12345 подвыборок
boot_x = IIDBootstrap(x, seed=123)
boot_x.conf_int(np.mean, method='basic', reps=12345, size=0.95)    # получили интервал для мат ожидания (мат ожидание будет принадлежать этому доверительному интервалу с вероятностью 95%)

array([[478.26      ],
       [922.44666667]])

Видно, что по генеральной совокурности средняя продолжительность поездки на такси составит от 478 секунд до 922 секунд.

In [14]:
boot_x.conf_int(np.std, method='basic', reps=12345, size=0.95)    # получили доверительный интервал в котором лежит ско

array([[ 276.64416597],
       [1016.10400266]])

Отклонение от среднего значения может быть от 276 секунд до 1016 секунд

Теперь оценик корреляцию между x - длительность поездки и w - количество пассажиров

In [18]:
w = rng.choice(df['passenger_count'], size=30)

In [19]:
np.corrcoef(x, w)

array([[1.        , 0.12082711],
       [0.12082711, 1.        ]])

По матрице корреляций между двумя признаками, видно, что зависимость между x и w минимальна

In [22]:
# выведем значение одного коэффициента корреляции
def corr(x,y):
    corr_mat = np.corrcoef(x,y)
    return corr_mat[0, 1]

In [21]:
corr(x, w)

0.12082711215317765

In [24]:
# Через пакет найдем интервал, в котором будет лежать истинное значение корреляции
boot_xw = IIDBootstrap(x, w, seed=123)
boot_xw.conf_int(corr, method='basic', reps=12345, size=0.95)

array([[-0.18539241],
       [ 0.3409211 ]])

Из полученного результата видно, что даже самая ма
ксимальная корреляция является незначимой, таким образом, корреляции между рассмотренными величинами нет - продолжительность поездки не зависит от количества пассажиров линейно.

In [38]:
data = pd.read_csv("/home/ksu/Загрузки/autos.csv")
rng = np.random.default_rng(123)
d = rng.choice(data['price'], size=38)
d

array([ 1890,  9900,  2400,  4990,  3600,   500, 26500,   544,  4000,
       17500, 15000,  6200,  2700,  9800,  3000,  1500,  1500, 13790,
        4500, 18650,  6200, 16500, 15800,  6990,  1200, 12250,   750,
        2600,  5500,  2200,  8900,  7000,  4125, 14690,  1600,   999,
        1200,   300])

d - цена на автомобили на вторичном рынке в Германии, х - данные о такси в Нью-. Так что предположим, что выборки независимы друг от друга (базы данных разные)

In [30]:
def mean_diff(x, y):
    return np.mean(y)-np.mean(x)

In [39]:
boot_xd = IndependentSamplesBootstrap(x,d, seed=123)
boot_xd.conf_int(mean_diff, reps=12345, size=0.95, method='basic')

array([[3952.54526316],
       [7930.04666667]])

Разница между настоящими мат ожиданиями этих двух выборок лежит в данном промежутке